# PmagPy structure

The PmagPy 

https://github.com/PmagPy/PmagPy/blob/master/CONTRIBUTING.md



# The command line programs

As an example, let's consider that we want to draw 10 random samples from a specified Fisher distribution. This can be done with the command line program **fishrot.py**. The command line programs takes flags




In [8]:
!fishrot.py -h

  File "/Users/Laurentia/PmagPy/programs/fishrot.py", line 33
    print main.__doc__
             ^
SyntaxError: Missing parentheses in call to 'print'


In [2]:
import pmagpy.pmag as pmag
import pmagpy.ipmag as ipmag

In [3]:
ipmag.fishrot?

In [6]:
ipmag.fishrot(k=20, n=10, dec=45, inc=30)

[[84.636956046144519, 30.741763200979346, 1.0],
 [56.252071804556181, 23.226205155587284, 1.0],
 [34.258457866266212, 37.769019005646747, 1.0],
 [354.7152969919066, 11.332353188782553, 1.0],
 [37.233840439804169, 34.53863604946347, 1.0],
 [9.5897109759770274, 28.054754222707324, 1.0],
 [56.579679066090989, 34.862547913028962, 1.0],
 [64.864200780476608, 18.395623635441474, 1.0],
 [16.841290506360508, 31.546095984821939, 1.0],
 [27.911824649842742, 42.544129175383027, 1.0]]